In [3]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os



In [4]:
data_path = os.path.join("..", "data")
print (f"Data path is: {data_path}")

Data path is: ../data


In [5]:
X = np.load(data_path + '/initial_data/function_1/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_1/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.31940389 0.76295937]
 [0.57432921 0.8798981 ]
 [0.73102363 0.73299988]
 [0.84035342 0.26473161]
 [0.65011406 0.68152635]
 [0.41043714 0.1475543 ]
 [0.31269116 0.07872278]
 [0.68341817 0.86105746]
 [0.08250725 0.40348751]
 [0.88388983 0.58225397]]
Y is :[ 1.32267704e-079  1.03307824e-046  7.71087511e-016  3.34177101e-124
 -3.60606264e-003 -2.15924904e-054 -2.08909327e-091  2.53500115e-040
  3.60677119e-081  6.22985647e-048]


In [6]:
# add the week 1 input and result to the array
X = np.append(X, [[.428571 , .142857 ]], axis=0)
y = np.append(y, [1.51922693823868E-55], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.775510, 0.795918]], axis=0)
y = np.append(y, [5.624127147502003e-35], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.808081, 0.717172]], axis=0)
y = np.append(y, [-1.588995756684446e-28], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.399405, 0.632402]], axis=0)
y = np.append(y, [-1.9365253523356726e-32], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.040981, 0.991637]], axis=0)
y = np.append(y, [0], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.272727, 0.646465]], axis=0)
y = np.append(y, [-2.4207951479406607e-51], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.343434, 0.686869]], axis=0)
y = np.append(y, [2.454153704078543e-54], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.31940389 0.76295937]
 [0.57432921 0.8798981 ]
 [0.73102363 0.73299988]
 [0.84035342 0.26473161]
 [0.65011406 0.68152635]
 [0.41043714 0.1475543 ]
 [0.31269116 0.07872278]
 [0.68341817 0.86105746]
 [0.08250725 0.40348751]
 [0.88388983 0.58225397]
 [0.428571   0.142857  ]
 [0.77551    0.795918  ]
 [0.808081   0.717172  ]
 [0.399405   0.632402  ]
 [0.040981   0.991637  ]
 [0.272727   0.646465  ]
 [0.343434   0.686869  ]]
Y is :[ 1.32267704e-079  1.03307824e-046  7.71087511e-016  3.34177101e-124
 -3.60606264e-003 -2.15924904e-054 -2.08909327e-091  2.53500115e-040
  3.60677119e-081  6.22985647e-048  1.51922694e-055  5.62412715e-035
 -1.58899576e-028 -1.93652535e-032  0.00000000e+000 -2.42079515e-051
  2.45415370e-054]


In [7]:
print (f"max y is :{np.max(y)}")

max y is :7.710875114502849e-16


In [8]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=2.5),
                         normalize_y=True)

In [9]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 7 changed kappa to 1 for exploitation
# week 8 changed kappa to 0.5 for more exploitation
def ucb(x, gp, kappa=0.5):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [10]:
grid_size = 100
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b] for a in x1 for b in x2])

In [11]:
ucb_values = np.array([ucb(x, gp) for x in X_grid])


In [12]:
best_idx = np.argmax(ucb_values)
next_best_point = X_grid[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Next best point to evaluate (UCB):", np.round(next_best_point,6))
print("Next best point to evaluate (UCB) without rounding:", next_best_point)
print("Predicted maximum value (mean):", pred_mean[0])

Next best point to evaluate (UCB): [0.757576 0.747475]
Next best point to evaluate (UCB) without rounding: [0.75757576 0.74747475]
Predicted maximum value (mean): 0.0002155276341727674


In [13]:



# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 2)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)

Best candidate input: [0.48943771 0.94311202]
Best candidate input rounded: [0.489438 0.943112]
Predicted maximum value: -0.0018030313221813477
